# **In this note book I'll be running Zephyr model**


---
<a target="_blank" href="https://colab.research.google.com/github/samsonthomas951/Large-Language-Model-Notebooks/blob/main/1.Running%20LLMs%20Locally/Zephyr_7b.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>

This is the essiest way I have ever runned a model locally using Python

As always I will start by installing the required libraries 
* Transformerss

* accelerate

* torch


In [ ]:
# Install transformers from source - only needed for versions <= v4.34
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-h56__f7h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-h56__f7h
  Resolved https://github.com/huggingface/transformers.git to commit c34c50cdc01ed5a5dc73a2f36466d07e6fe9fb4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7881253 sha256=5bff5b897d08c2bac77aa5cc2aed0f8e80f8e18b8285908a059b0d99c13d6a99
  Stored in directory: /tmp/pip-ephem-whee

In [ ]:
import torch
from transformers import pipeline

Specify **Zephyr-7b-alpha model** to be downloaded via the pipeline 

In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

I'll add the chat template an the system prompt form [here](https://huggingface.co/docs/transformers/main/en/chat_templating)

In [ ]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly and helpful assistant",
    },
]

In the below cell I'll add the the role of the user "you" and the content "Prompt" to message dictionary before passing it to the model.
This is an improvement form the original, hich required modification of the content everytime.

```python 

messages = [
{
"role": "system",
"content": "You are a friendly and helpful assistant",
},
{"role": "user", "content": "Define life"},
]

```

In [ ]:
while True:
    user_input = input("Enter your message: ")
    messages.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    print(outputs[0]["generated_text"])
    if user_input.lower() == "quit":
        break

## Adding memory to the model

---


The below cell adds the memory to the message hence you can have a conversation with the model via long context of **8K** or **8000 Tokens**, Please note that when you exceed this limit the the session, model might crash and it will require you to restart the runtime and rerun every cell in the note book. 

In [ ]:
while True:
    user_input = input("Enter your message: ")
    messages.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    generated_text = outputs[0]["generated_text"]
    print(generated_text)
    messages.append({"role": "assistant", "content": generated_text})  # Append the model's output to the messages list
    if user_input.lower() == "quit":
        break